In [8]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from utilsCPD import *
import matplotlib.pyplot as plt
from SWCPD import BaseDetector as SWDCP
import time


BATCH_SIZE  =32
mnist_transforms = transforms.Compose([transforms.ToTensor()])
train_val_dataset = datasets.MNIST(root="./datasets/", train=True, download=True, transform=mnist_transforms)
test_dataset = datasets.MNIST(root="./datasets/", train=False, download=True, transform=mnist_transforms)
train_size = int(0.9 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset=train_val_dataset, lengths=[train_size, val_size])
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
all_batches = []
all_labels =[]
torch.manual_seed(10)
for x,y in test_dataloader:
    bs = x.shape[0]
    all_batches.append(x.flatten().reshape(bs,784))
    all_labels.append(y)

all_batches = torch.cat(all_batches,dim=0)
all_labels = torch.cat(all_labels,dim=0)

print(all_batches.shape)

torch.Size([10000, 784])


In [10]:
Labels_Sampled = []
seed = 2025
rng = np.random.default_rng(seed)


def sample_classes(seed,n_classes = 3,nb_experiments=5):
    rng = np.random.default_rng(seed)
    Labels_Sampled = []
    for run in range(nb_experiments):
        l = tuple(rng.choice(np.arange(10),n_classes,replace=False))
        if l not in Labels_Sampled:
            Labels_Sampled.append(l)
    return Labels_Sampled

Labels_Sampled = sample_classes(seed=seed, n_classes=3,nb_experiments=5)

print(Labels_Sampled)
def FilterData(all_batches,all_labels,lables,N_Samples=200):
    data = []
    cps = []
    for l in lables:
        d =all_batches[all_labels==l].detach().numpy()
        data.append(d[:N_Samples])
        cps.append(len(d[:N_Samples]))
    return np.vstack(data),np.cumsum(cps)[:-1]


FilterData(all_batches,all_labels,list(Labels_Sampled[0]))


[(3, 9, 8), (6, 5, 8), (0, 4, 3), (6, 7, 8), (4, 3, 0)]


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([200, 400]))

In [4]:
LABELS_Sampled = [sample_classes(seed=seed,n_classes=3,nb_experiments=5),sample_classes(seed=seed, n_classes=4,nb_experiments=5),sample_classes(seed=seed, n_classes=5,nb_experiments=5)]
LABELS_Sampled

[[(3, 9, 8), (6, 5, 8), (0, 4, 3), (6, 7, 8), (4, 3, 0)],
 [(3, 7, 8, 9), (8, 3, 6, 5), (7, 6, 3, 8), (8, 7, 3, 2), (6, 7, 4, 2)],
 [(2, 6, 3, 7, 9),
  (5, 2, 3, 4, 0),
  (4, 0, 3, 8, 2),
  (4, 6, 7, 0, 2),
  (0, 1, 9, 3, 2)]]

In [67]:
METRICS_SWD = {}
METRICS_SWD_5 = {}
METRICS_SWD_10 = {}
METRICS_SWD_15 = {}
METRICS_SWD_30 = {}

Data_OUT = {}
LABELS_Sampled = [sample_classes(seed=seed,n_classes=3,nb_experiments=5),sample_classes(seed=seed, n_classes=4,nb_experiments=5),sample_classes(seed=seed, n_classes=5,nb_experiments=5)]
ind = 0
ind2 = 0
LABELS_Sampled
for id,i in enumerate(range(15)):
    print(id)
    Data_OUT[id] = {}
    METRICS_SWD[id]  = {}
    METRICS_SWD_5[id] = {}
    METRICS_SWD_10[id] = {}
    METRICS_SWD_15[id] = {}
    METRICS_SWD_30[id] = {}
    if i in [5,10]:
        ind+=1
        ind2 = 0
    df, GroundTruth = FilterData(all_batches,all_labels,list(LABELS_Sampled[ind][ind2]),N_Samples=200)
    Data_OUT[id]['data'] = df
    Data_OUT[id]['target'] = GroundTruth
    ind2+=1
    print(GroundTruth)
    detector = SWDCP(data=df,window_length=50,max_history=25,significance=0.1)
    print(detector.device)
    start_time = time.time()
    detector.process_dataloader(p=4,n_theta=5000,seed=2025)
    end_time = time.time()
    f1, covering_score, auc_score,fp,delay = detector.evaluate(GroundTruth,20)
    METRICS_SWD[id]['F1'] = f1
    METRICS_SWD[id]['Covering'] = covering_score
    METRICS_SWD[id]['AUC'] = auc_score
    METRICS_SWD[id]['DD'] = delay[1]
    METRICS_SWD[id]['FP'] = fp
    METRICS_SWD[id]['RunTime'] = end_time -start_time

    f1, covering_score, auc_score,fp,delay = detector.evaluate(GroundTruth,5)
    METRICS_SWD_5[id]['AUC'] = auc_score
    METRICS_SWD_5[id]['FP'] = fp
    f1, covering_score, auc_score,fp,delay = detector.evaluate(GroundTruth,10)
    METRICS_SWD_10[id]['AUC'] = auc_score
    METRICS_SWD_10[id]['FP'] = fp
    f1, covering_score, auc_score,fp,delay = detector.evaluate(GroundTruth,15)
    METRICS_SWD_15[id]['AUC'] = auc_score
    METRICS_SWD_15[id]['FP'] = fp
    f1, covering_score, auc_score,fp,delay = detector.evaluate(GroundTruth,30)
    METRICS_SWD_30[id]['AUC'] = auc_score
    METRICS_SWD_30[id]['FP'] = fp
    print(METRICS_SWD[id])

0
[200 400]
cuda


 32%|███▏      | 174/551 [00:04<00:08, 44.19it/s]

Change detected at: 214 
Initiate new segment


 68%|██████▊   | 372/551 [00:08<00:03, 58.04it/s]

Change detected at: 415 
Initiate new segment


...

Change detected at: 612 
Initiate new segment


 81%|████████  | 771/951 [00:16<00:04, 44.94it/s]

Change detected at: 812 
Initiate new segment


100%|██████████| 951/951 [00:20<00:00, 46.83it/s]

F1 score: 0.9090909090909091
Covering: 0.9066850507982583
0.9166666666666667
F1 score: 0.1818181818181818
Covering: 0.9066850507982583
0.18333333333333335
F1 score: 0.5454545454545454
Covering: 0.9066850507982583
0.5499999999999999
F1 score: 0.9090909090909091
Covering: 0.9066850507982583
0.9166666666666667
F1 score: 0.9090909090909091
Covering: 0.9066850507982583
0.9166666666666667
{'F1': 0.9090909090909091, 'Covering': 0.9066850507982583, 'AUC': 0.9166666666666667, 'DD': 12.8, 'FP': 1, 'RunTime': 20.316514492034912}


In [6]:
METRICS_SWD

{0: {'F1': 1.0,
  'Covering': 0.9082319061073679,
  'AUC': 1.0,
  'DD': 14.5,
  'FP': 0,
  'RunTime': 8.714927434921265},
 1: {'F1': 0.8,
  'Covering': 0.6467338217338217,
  'AUC': 0.8333333333333333,
  'DD': 10.0,
  'FP': 0,
  'RunTime': 11.508496284484863},
 2: {'F1': 1.0,
  'Covering': 0.9357142857142857,
  'AUC': 1.0,
  'DD': 10.0,
  'FP': 0,
  'RunTime': 12.78207540512085},
 3: {'F1': 1.0,
  'Covering': 0.9357329462989841,
  'AUC': 1.0,
  'DD': 10.0,
  'FP': 0,
  'RunTime': 12.451367378234863},
 4: {'F1': 1.0,
  'Covering': 0.9482370164894436,
  'AUC': 1.0,
  'DD': 8.0,
  'FP': 0,
  'RunTime': 12.497658491134644},
 5: {'F1': 0.75,
  'Covering': 0.898812041562803,
  'AUC': 0.75,
  'DD': 14.333333333333334,
  'FP': 1,
  'RunTime': 17.414814472198486},
 6: {'F1': 1.0,
  'Covering': 0.9301071591427268,
  'AUC': 1.0,
  'DD': 9.666666666666666,
  'FP': 0,
  'RunTime': 17.21961498260498},
 7: {'F1': 1.0,
  'Covering': 0.9165552091567757,
  'AUC': 1.0,
  'DD': 11.666666666666666,
  'FP': 

In [12]:
f1,cov, AUC, DD, FP, RT = [],[], [], [],[],[]
for k,metrics in METRICS_SWD.items():
    f1.append(metrics['F1'])
    cov.append(metrics['Covering'])
    AUC.append(metrics['AUC'])
    DD.append(metrics['DD'])
    FP.append(metrics['FP'])
    RT.append(metrics['RunTime'])

print('F1',np.mean(f1),np.std(f1))
print('Cov',np.mean(cov),np.std(cov))
print('AUC',np.mean(AUC),np.std(AUC))
print('DD',np.mean(DD),np.max(DD),np.min(DD))
print('FP',np.mean(FP),np.std(FP),np.max(FP),np.min(FP))
print('RunTime',np.mean(RT),np.std(RT))


F1 0.963939393939394 0.07799955263463336
Cov 0.8974051289243116 0.06873350569680424
AUC 0.9666666666666666 0.073282810879294
DD 11.82 14.5 8.0
FP 0.13333333333333333 0.33993463423951903 1 0
RunTime 18.17991789182027 4.559825638412763


In [66]:
METRICS_SWD_15

{0: {'AUC': 0},
 1: {'AUC': 0},
 2: {'AUC': 0},
 3: {'AUC': 0},
 4: {'AUC': 0},
 5: {'AUC': 1},
 6: {'AUC': 0},
 7: {'AUC': 0},
 8: {'AUC': 0},
 9: {'AUC': 1},
 10: {'AUC': 2},
 11: {'AUC': 2},
 12: {'AUC': 0},
 13: {'AUC': 0},
 14: {'AUC': 1}}

In [70]:
def sum_aucs(METRICS_SWD):
    AUC = []
    FP = []
    for k,metrics in METRICS_SWD.items():
        #print(metrics)
        AUC.append(metrics['AUC'])
        FP.append(metrics['FP'])

    return np.mean(AUC),np.mean(FP)




SWD_AUC = [sum_aucs(d)[0] for d in [METRICS_SWD_5,METRICS_SWD_10,METRICS_SWD_15,METRICS_SWD,METRICS_SWD_30]]
SWD_FPS = [sum_aucs(d)[1] for d in [METRICS_SWD_5,METRICS_SWD_10,METRICS_SWD_15,METRICS_SWD,METRICS_SWD_30]]

SWD_FPS

[2.933333333333333,
 1.6666666666666667,
 0.4666666666666667,
 0.13333333333333333,
 0.0]

In [ ]:
# import json

# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super().default(obj)
    
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(Data_OUT, f, ensure_ascii=False, indent=4,cls=NumpyEncoder)

In [74]:
from claspy.segmentation import BinaryClaSPSegmentation
ind = 0
ind2 = 0
METRICS_Claspy = {}
METRICS_Claspy_5 = {}
METRICS_Claspy_10 = {}
METRICS_Claspy_15 = {}
METRICS_Claspy_30 = {}

for id,i in enumerate(range(15)):
    print(id)
    METRICS_Claspy[id]  = {}
    METRICS_Claspy_5[id] = {}
    METRICS_Claspy_10[id] = {}
    METRICS_Claspy_15[id] = {}
    METRICS_Claspy_30[id] = {}
    if i in [5,10]:
        ind+=1
        ind2 = 0
    df, GroundTruth = FilterData(all_batches,all_labels,list(LABELS_Sampled[ind][ind2]),N_Samples=200)
    ind2+=1
    print(GroundTruth)
    start_time = time.time()
    clasp = BinaryClaSPSegmentation(window_size=100).fit(df.astype(np.float64))
    f1,auc_score,fp = f_measure({'0':GroundTruth},clasp.change_points,20)
    covering_score = covering({'0':GroundTruth},clasp.change_points,df.shape[0])
    end_time =time.time()
    print(end_time-start_time)
    METRICS_Claspy[id]['F1'] = f1
    METRICS_Claspy[id]['Covering'] = covering_score
    METRICS_Claspy[id]['AUC'] = auc_score
    METRICS_Claspy[id]['DD'] = detection_delay(GroundTruth,clasp.change_points)[1]
    METRICS_Claspy[id]['FP'] = fp
    METRICS_Claspy[id]['RunTime'] = end_time- start_time


    f1,auc_score,fp = f_measure({'0':GroundTruth},clasp.change_points,5)
    METRICS_Claspy_5[id]['AUC'] = auc_score
    METRICS_Claspy_5[id]['FP'] = fp

    f1,auc_score,fp = f_measure({'0':GroundTruth},clasp.change_points,10)
    METRICS_Claspy_10[id]['AUC'] = auc_score
    METRICS_Claspy_10[id]['FP'] = fp

    f1,auc_score,fp = f_measure({'0':GroundTruth},clasp.change_points,15)
    METRICS_Claspy_15[id]['AUC'] = auc_score
    METRICS_Claspy_15[id]['FP'] = fp

    f1,auc_score,fp = f_measure({'0':GroundTruth},clasp.change_points,30)
    METRICS_Claspy_30[id]['AUC'] = auc_score
    METRICS_Claspy_30[id]['FP'] = fp

    print(METRICS_Claspy[id])

0
[200 400]
0.001477956771850586
{'F1': 0.5, 'Covering': 0.33333333333333326, 'AUC': 0.6666666666666667, 'DD': 0, 'FP': 0, 'RunTime': 0.001477956771850586}
1
[200 400]
0.0010204315185546875
{'F1': 0.5, 'Covering': 0.33333333333333326, 'AUC': 0.6666666666666667, 'DD': 0, 'FP': 0, 'RunTime': 0.0010204315185546875}
2
[200 400]
0.0010373592376708984
{'F1': 0.5, 'Covering': 0.33333333333333326, 'AUC': 0.6666666666666667, 'DD': 0, 'FP': 0, 'RunTime': 0.0010373592376708984}
3
[200 400]
0.0009996891021728516
{'F1': 0.5, 'Covering': 0.33333333333333326, 'AUC': 0.6666666666666667, 'DD': 0, 'FP': 0, 'RunTime': 0.0009996891021728516}
4
[200 400]
0.0020127296447753906
{'F1': 0.5, 'Covering': 0.33333333333333326, 'AUC': 0.6666666666666667, 'DD': 0, 'FP': 0, 'RunTime': 0.0020127296447753906}
5
[200 400 600]
0.002000570297241211
{'F1': 0.4, 'Covering': 0.25, 'AUC': 0.625, 'DD': 0, 'FP': 0, 'RunTime': 0.002000570297241211}
6
[200 400 600]
0.0030684471130371094
{'F1': 0.4, 'Covering': 0.25, 'AUC': 0.625

c:\Users\Sven Jacob\miniconda3\envs\torch\Lib\site-packages\claspy\segmentation.py:212: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(
c:\Users\Sven Jacob\miniconda3\envs\torch\Lib\site-packages\claspy\segmentation.py:212: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(
c:\Users\Sven Jacob\miniconda3\envs\torch\Lib\site-packages\claspy\segmentation.py:212: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(
c:\Users\Sven Jacob\miniconda3\envs\torch\Lib\site-packages\claspy\segmentation.py:212: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(
c:\Users\Sven Jacob\miniconda3\envs\torch\Lib\site-packages\claspy\segmentation.py:2

3.5052425861358643
{'F1': 0.33333333333333337, 'Covering': 0.2, 'AUC': 0.6000000000000001, 'DD': 0, 'FP': 0, 'RunTime': 3.5052425861358643}
11
[200 400 600 800]
3.3517096042633057
{'F1': 0.33333333333333337, 'Covering': 0.2, 'AUC': 0.6000000000000001, 'DD': 0, 'FP': 0, 'RunTime': 3.3517096042633057}
12
[200 400 600 800]
3.4377753734588623
{'F1': 0.33333333333333337, 'Covering': 0.2, 'AUC': 0.6000000000000001, 'DD': 0, 'FP': 0, 'RunTime': 3.4377753734588623}
13
[200 400 600 800]
3.4332096576690674
{'F1': 0.33333333333333337, 'Covering': 0.2, 'AUC': 0.6000000000000001, 'DD': 0, 'FP': 0, 'RunTime': 3.4332096576690674}
14
[200 400 600 800]
3.463819742202759
{'F1': 0.33333333333333337, 'Covering': 0.2, 'AUC': 0.6000000000000001, 'DD': 0, 'FP': 0, 'RunTime': 3.463819742202759}


In [75]:
AUC_CLASP = [sum_aucs(d)[0] for d in [METRICS_Claspy_5,METRICS_Claspy_10,METRICS_Claspy_15,METRICS_Claspy,METRICS_Claspy_30]]
FP_CLASP = [sum_aucs(d)[1] for d in [METRICS_Claspy_5,METRICS_Claspy_10,METRICS_Claspy_15,METRICS_Claspy,METRICS_Claspy_30]]
FP_CLASP

[0.0, 0.0, 0.0, 0.0, 0.0]

In [20]:
f1_CL,cov_CL, AUC_CL, DD_CL, FP_CL,RT_CL = [],[], [], [],[],[]
for k,metrics in METRICS_Claspy.items():
    f1_CL.append(metrics['F1'])
    cov_CL.append(metrics['Covering'])
    AUC_CL.append(metrics['AUC'])
    DD_CL.append(metrics['DD'])
    FP_CL.append(metrics['FP'])
    RT_CL.append(metrics['RunTime'])

print('F1',np.mean(f1_CL),np.std(f1_CL))
print('Cov',np.mean(cov_CL),np.std(cov_CL))
print('AUC',np.mean(AUC_CL),np.std(AUC_CL))
print('DD',np.mean(DD_CL),np.max(DD_CL),np.min(DD_CL))
print('FP',np.mean(FP_CL),np.std(FP_CL),np.max(FP_CL),np.min(FP_CL))
print('RT',np.mean(RT_CL),np.std(RT_CL))


F1 0.4111111111111111 0.0684934889218775
Cov 0.2611111111111111 0.05499719409228699
AUC 0.6305555555555554 0.027498597046143512
DD 0.0 0 0
FP 0.0 0.0 0 0
RT 1.1770958264668783 1.6624028762695355


In [53]:
np.std(RT_CL[10:])

0.06754145648587327

In [35]:
data_out = {}
ind2=0
ind=0
for id,i in enumerate(range(15)):
    if i in [5,10]:
        ind+=1
        ind2 = 0
    df, GroundTruth = FilterData(all_batches,all_labels,list(LABELS_Sampled[ind][ind2]),N_Samples=200)
    data_out[id] = {'data':df,'labels':GroundTruth}

In [22]:
# import json

# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super().default(obj)
    
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(data_out, f, ensure_ascii=False, indent=4,cls=NumpyEncoder)

### Results for E-divisive and BOCPD

In [52]:
import json

with open("./R/res//MNIST_Edivise.json") as f:
    ECP_Results = json.load(f)


F1,Covering, AUC,FP,DD,RT = [],[],[],[],[],[]

for id, res in ECP_Results.items():
    print(res)
    cps = res#['CPs']
    ground_truths = data_out[int(id)]['labels']
    n = ground_truths[-1] + 200
    cps = cps[1:-1]

    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    delay = detection_delay(ground_truths,cps)[1]
    F1.append(f1)
    AUC.append(AUC_score)
    Covering.append(covering_score)
    FP.append(fp)
    DD.append(delay)
    #RT.append(res['RunTime'])


print(np.mean(F1))
print(np.std(Covering))
print(np.mean(AUC))

print('DD',np.mean(DD),np.min(DD),np.max(DD))
print('FP',np.mean(FP),np.min(FP),np.max(FP))
#print('RT',np.mean(RT),np.std(RT))



[1, 201, 401, 601]
[1, 201, 401, 601]
[1, 201, 401, 601]
[1, 201, 401, 601]
[1, 201, 401, 601]
[1, 201, 288, 401, 601, 801]
[1, 201, 288, 401, 601, 801]
[1, 201, 288, 401, 601, 801]
[1, 201, 288, 401, 601, 801]
[1, 201, 288, 401, 601, 801]
[1, 201, 401, 601, 801, 1001]
[1, 201, 401, 601, 688, 801, 1001]
[1, 201, 401, 601, 801, 1001]
[1, 201, 401, 601, 801, 1001]
[1, 201, 401, 601, 801, 1001]
0.9569023569023568
0.05164545795524169
0.961111111111111
DD 9.41 1.0 22.75
FP 0.4 0 1


In [41]:
with open("./R/res/MNIST_BOCPD_new.json") as f:
    ECP_Results = json.load(f)


F1,Covering, AUC,FP,DD,RT = [],[],[],[],[],[]

for id, res in ECP_Results.items():
    #print(cps)
    ground_truths = data_out[int(id)]['labels']
    n = ground_truths[-1] + 200
    cps = res['CPs']
    cps = cps[0][1:]
    #print(cps)
    f1, AUC_score,fp = f_measure({'0':ground_truths},cps,20)
    covering_score = covering({'0':ground_truths},cps,n)
    delay = detection_delay(ground_truths,cps)[1]
    F1.append(f1)
    AUC.append(AUC_score)
    Covering.append(covering_score)
    FP.append(fp)
    DD.append(delay)
    RT.append(res['RunTime'])

print(np.mean(F1))
print(np.mean(Covering))
print(np.mean(AUC))

print('DD',np.mean(DD),np.min(DD),np.max(DD))
print('FP',np.mean(FP),np.min(FP),np.max(FP))
print('RT',np.mean(RT),np.std(RT))

0.6844444444444445
0.7847463171351448
0.6916666666666668
DD 17.87777777777778 11.0 27.0
FP 0.9333333333333333 0 2
RT 17.357360000000003 22.77953078582612


In [78]:
import json

def GET_AUCs_ECP(tau=100):
    with open("./R/res//MNIST_Edivise.json") as f:
        ECP_Results = json.load(f)


    F1,Covering, AUC,FP,DD,RT = [],[],[],[],[],[]

    for id, res in ECP_Results.items():
        cps = res#['CPs']
        ground_truths = data_out[int(id)]['labels']
        n = ground_truths[-1] + 200
        cps = cps[1:-1]

        f1, AUC_score,fp = f_measure({'0':ground_truths},cps,tau)
        covering_score = covering({'0':ground_truths},cps,n)
        delay = detection_delay(ground_truths,cps)[1]
        F1.append(f1)
        AUC.append(AUC_score)
        Covering.append(covering_score)
        FP.append(fp)
        DD.append(delay)
       # RT.append(res['RunTime'])
    return np.mean(AUC),np.mean(FP)

    
    

def GET_AUCs_BOCPD(tau=100):
    with open("./R/res/MNIST_BOCPD_new.json") as f:
        ECP_Results = json.load(f)


    F1,Covering, AUC,FP,DD,RT = [],[],[],[],[],[]

    for id, res in ECP_Results.items():
        #print(cps)
        ground_truths = data_out[int(id)]['labels']
        n = ground_truths[-1] + 200
        cps = res['CPs']
        cps = cps[0][1:]
        #print(cps)
        f1, AUC_score,fp = f_measure({'0':ground_truths},cps,tau)
        covering_score = covering({'0':ground_truths},cps,n)
        delay = detection_delay(ground_truths,cps)[1]
        F1.append(f1)
        AUC.append(AUC_score)
        Covering.append(covering_score)
        FP.append(fp)
        DD.append(delay)
        RT.append(res['RunTime'])

    return np.mean(AUC),np.mean(FP)



GET_AUCs_ECP(200)


ECPS_M = [GET_AUCs_ECP(t)[0] for t in [5,10,15,20,30]]
BOCPD_M = [GET_AUCs_BOCPD(t)[0] for t in [5,10,15,20,30]]

print(ECPS_M)
print(BOCPD_M)


ECPS_M_FP = [GET_AUCs_ECP(t)[1] for t in [5,10,15,20,30]]
BOCPD_M_FP = [GET_AUCs_BOCPD(t)[1] for t in [5,10,15,20,30]]

print(ECPS_M_FP)
print(BOCPD_M_FP)

[0.961111111111111, 0.961111111111111, 0.961111111111111, 0.961111111111111, 0.961111111111111]
[0.2838888888888889, 0.46388888888888885, 0.6199999999999999, 0.6916666666666668, 0.8783333333333335]
[0.4, 0.4, 0.4, 0.4, 0.4]
[2.533333333333333, 1.8, 1.2, 0.9333333333333333, 0.26666666666666666]


In [41]:
print('RT',np.mean(np.vstack((RT[:5],np.array(RT[5:])*60))),np.std(np.vstack((RT[:5],np.array(RT[5:])*60))))

RT 69.32691999999999 16.61958226102369


### Results


| METHOD  | DD (min;max) | FP (min;max) |
|-------------|----------|-----------|
| SWD (ours)        | 15.5 (10.0;20.5)  | 0.46 (0;2) |
| ECP         | 9.41 (0;22.75) | 0.4 (0;1)  |
| KCPA         |21.7 (0;70.75) | 0.66 (0;2) |
| CLasP         | 0 (0;0) |0 (0;0) |
| BOCPD         | 17.8 (11;27)  | 0.93 (0;2) |